# Setup
Quick setup example.

(For more information on what is available with the Hierarchical Localization codebase specifically, please check out the original notebooks from their repository: https://github.com/cvg/Hierarchical-Localization)

In [ ]:
images = Path("datasets/put_your_data_folder_here")
outputs = Path("outputs/put_your_desired_output_folder_name_here/")
# !rm -rf $outputs <-- This line is useful if you want to clear up your outputs
sfm_pairs = outputs / "pairs-sfm.txt"
loc_pairs = outputs / "pairs-loc.txt"
sfm_dir = outputs / "sfm"
features = outputs / "features.h5"
matches = outputs / "matches.h5"

#These two lines decide which feature extraction and matching methods you want to use
feature_conf = extract_features.confs["disk"]
matcher_conf = match_features.confs["disk+lightglue"]

# Example
An example of a full SfM reconstruction setup.

In [ ]:
images = Path("datasets/put_your_data_folder_here")
outputs = Path("outputs/put_your_desired_output_folder_name_here/")
# !rm -rf $outputs <-- This line is useful if you want to clear up your outputs
sfm_pairs = outputs / "pairs-sfm.txt"
loc_pairs = outputs / "pairs-loc.txt"
sfm_dir = outputs / "sfm"
features = outputs / "features.h5"
matches = outputs / "matches.h5"

#These two lines decide which feature extraction and matching methods you want to use
feature_conf = extract_features.confs["disk"]
matcher_conf = match_features.confs["disk+lightglue"]

my_references = [p.relative_to(images).as_posix() for p in sorted((images).iterdir())]

#Extract features
extract_features.main(
    feature_conf, images, image_list=my_references, feature_path=features
)

#Create pairs and then matchings
pairs_from_exhaustive.main(sfm_pairs, image_list=my_references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

#Create a reconstruction with the desired options. Here we are assuming a single camera is used (CameraMode.SINGLE) and that our camera is of the SIMPLE_RADIAL model.
model = reconstruction.main(
    sfm_dir, images, sfm_pairs, features, matches, image_list=my_references, camera_mode=pycolmap.CameraMode.SINGLE, image_options=dict(camera_model='SIMPLE_RADIAL')
)

#Save the result in text
model.write_text(os.getcwd()/outputs)

#Export a point cloud of the reconstruction
model.export_PLY(os.path.join(os.getcwd()/outputs,"rec.ply"))

# Visualization
How to visualize our reconstruction (This can also be done by importing the previously exported point cloud with MeshLab).

In [ ]:
#Load the existing reconstruction
my_model = pycolmap.Reconstruction(os.getcwd()/outputs)

#Setup, plot and show the figure
fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig, my_model, color="rgba(255,0,0,0.5)", name="mug", points_rgb=True
)
fig.show()

# Example of MVS and Poisson Reconstruction
Example of getting a dense point cloud and then using Poisson Reconstruction for a surface reconstruction.

In [ ]:
images = Path("datasets/put_your_data_folder_here")
outputs = Path("outputs/put_your_desired_output_folder_name_here/")
# !rm -rf $outputs <-- This line is useful if you want to clear up your outputs
sfm_pairs = outputs / "pairs-sfm.txt"
loc_pairs = outputs / "pairs-loc.txt"
sfm_dir = outputs / "sfm"
features = outputs / "features.h5"
matches = outputs / "matches.h5"

#These two lines decide which feature extraction and matching methods you want to use
feature_conf = extract_features.confs["disk"]
matcher_conf = match_features.confs["disk+lightglue"]

my_references = [p.relative_to(images).as_posix() for p in sorted((images).iterdir())]

#Extract features
extract_features.main(
    feature_conf, images, image_list=my_references, feature_path=features
)

#Create pairs and then matchings
pairs_from_exhaustive.main(sfm_pairs, image_list=my_references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

#Create a reconstruction with the desired options. Here we are assuming a single camera is used (CameraMode.SINGLE) and that our camera is of the SIMPLE_RADIAL model.
model = reconstruction.main(
    sfm_dir, images, sfm_pairs, features, matches, image_list=my_references, camera_mode=pycolmap.CameraMode.SINGLE, image_options=dict(camera_model='SIMPLE_RADIAL')
)

#Save the result in text
model.write_text(os.getcwd()/outputs)

#Export a point cloud of the reconstruction
model.export_PLY(os.path.join(os.getcwd()/outputs,"rec.ply"))

#Create undistorted versions of our images, based on the camera model we are using to model the distortion
pycolmap.undistort_images(mvs_path, outputs, images)

#Do patch matching (extract depth maps) in before fusing information 
pycolmap.patch_match_stereo(mvs_path)  # Requires compilation with CUDA. This means that you need COLMAP installed from source, and you should be using the pycolmap code found in the COLMAP folder.

#Do fusion and export a dense point cloud
pycolmap.stereo_fusion(mvs_path / "dense.ply", mvs_path)

#Take the dense point cloud and create a mesh through Poisson Reconstruction
pycolmap.poisson_meshing(mvs_path / "dense.ply", mvs_path/ "poisson.ply")